In [10]:
import scala.util.matching
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.{min, max, mean, stddev, typedLit}
import org.apache.spark.sql.types.{IntegerType, DoubleType}
//import spark.implicits._ 

println("all good")

all good


data is in the format:

{
"conversionSymbol": "",
"volumeto": 6685400.72, 
"high": 9165.89, 
"low": 9144.6, 
"time": 1595030400, 
"volumefrom": 730.39, 
"close": 9151.68, 
"open": 9156.79, 
"conversionType": "direct"
}


In [11]:
case class TradingInfo (time: Int,
                        high: Double,
                        low: Double,
                        open: Double,
                        close: Double,
                        volumefrom: Double,
                        volumeto: Double, 
                        conversionType: String,
                        conversionSymbol: String                        
                       )

def createTradingInfo(data: String): TradingInfo = {
    val s = data.split(", ")
    return TradingInfo(s(0).split(": ")(1).toInt, s(1).split(": ")(1).toDouble, s(2).split(": ")(1).toDouble, 
                       s(3).split(": ")(1).toDouble, s(4).split(": ")(1).toDouble, s(5).split(": ")(1).toDouble, 
                       s(6).split(": ")(1).toDouble, s(7).split(": ")(1), s(8).split(": ")(1))
}

val filename = "../Datasets/bitcoin_json.txt"
//val johnFileName = "//home/john/ID2221-Labs/PreProcessing/"
val tradingInformation = sc.textFile(filename).
            flatMap(l => l.split("},")).
            //map(x => x.replace("\\", "")).
            map(x => x.replace("[", "")).
            map(x => x.replace("{", "")).
            map(x => x.replace("\"", "")).
            map(createTradingInfo)
            .cache()

tradingInformation.take(5).foreach(println)

println("All good")

TradingInfo(1595376000,9397.25,9365.01,9392.83,968.54,9082321.43,9366.13,\direct\,\\)
TradingInfo(1595379600,9391.11,9361.69,9366.13,782.86,7335886.8,9375.34,\direct\,\\)
TradingInfo(1595383200,9381.06,9365.19,9375.34,519.78,4869598.42,9369.76,\direct\,\\)
TradingInfo(1595386800,9375.95,9364.95,9369.76,411.52,3854451.45,9367.03,\direct\,\\)
TradingInfo(1595390400,9371.22,9356.88,9367.03,542.09,5075388.36,9356.94,\direct\,\\)
All good


defined class TradingInfo
filename = ../Datasets/bitcoin_json.txt
tradingInformation = MapPartitionsRDD[94] at map at <console>:47


createTradingInfo: (data: String)TradingInfo


MapPartitionsRDD[94] at map at <console>:47

In [12]:
// Data Action


// val df = tradingInformation.toDF("conversionSymbol","volumeto", "high", "low", "time", 
//                                  "volumefrom", "close","open","conversionType").show(5)




val df = spark.createDataFrame(tradingInformation).sort("time")
val df_1 = df.withColumn("midPrice",($"high" + $"low")/2)

val statistics = df_1.agg(mean("midPrice"), stddev("midPrice"), min("midPrice"), max("midPrice"),
                         mean("volumefrom"), stddev("volumefrom"), min("volumefrom"), max("volumefrom"),
                         mean("volumeto"), stddev("volumeto"), min("volumeto"), max("volumeto")).head()

val midPrice_mean = statistics.getDouble(0)
val midPrice_std = statistics.getDouble(1)
val midPrice_min = statistics.getDouble(2)
val midPrice_max = statistics.getDouble(3)
val midPrice_range = midPrice_max - midPrice_min

// VolumeFrom
val volumefrom_mean = statistics.getDouble(4)
val volumefrom_std = statistics.getDouble(5)
val volumefrom_min = statistics.getDouble(6)
val volumefrom_max = statistics.getDouble(7)
val volumefrom_range = volumefrom_max - volumefrom_min

// VolumeTo
val volumeto_mean = statistics.getDouble(8)
val volumeto_std = statistics.getDouble(9)
val volumeto_min = statistics.getDouble(10)
val volumeto_max = statistics.getDouble(11)
val volumeto_range = volumeto_max - volumeto_min

// from -1 to 1 
//val df_2 = df_1.withColumn("midPriceNorm",($"midPrice" - midPrice_mean)/midPrice_std)
// from 0 to 1 
val df_2 = df_1.withColumn("midPriceNorm",(($"midPrice" - midPrice_min)/midPrice_range).cast(DoubleType))
               .withColumn("volumefromNorm",(($"volumefrom" - volumefrom_min)/volumefrom_range).cast(DoubleType))
               .withColumn("volumetoNorm",(($"volumeto" - volumeto_min)/volumeto_range).cast(DoubleType))
               .cache()

df_2.show(5)
df_2.printSchema()

println("All good")

+----------+-------+------+-------+------+----------+--------+--------------+----------------+-----------------+--------------------+--------------------+--------------------+
|      time|   high|   low|   open| close|volumefrom|volumeto|conversionType|conversionSymbol|         midPrice|        midPriceNorm|      volumefromNorm|        volumetoNorm|
+----------+-------+------+-------+------+----------+--------+--------------+----------------+-----------------+--------------------+--------------------+--------------------+
|1388581200|  967.0|718.46|  812.9| 1.012|    889.99|   967.0|      \direct\|              \\|           842.73| 0.03430077268445006|3.860005357360329E-7|0.040873380340124756|
|1388584800| 759.11|750.13| 750.13| 1.661|   1282.36|  759.11|      \direct\|              \\|           754.62|0.029799236920461117|5.561766390706178E-7|0.030277484141069302|
|1388588400|1169.16|912.66|1169.16|0.5916|    575.07| 1000.26|      \direct\|              \\|          1040.91| 0.04442

df = [time: int, high: double ... 7 more fields]
df_1 = [time: int, high: double ... 8 more fields]
statistics = [4097.0546260549845,4112.281709973773,171.35000000000002,19744.67,1.243764883985811E7,2.930919029022742E7,0.0,2.30567037505E9,4097.705743962565,4113.609891509019,165.07,19784.93]
midPrice_mean = 4097.0546260549845
midPrice_std = 4112.281709973773
midPrice_min = 171.35000000000002
midPrice_max = 19744.67
midPrice_range = 19573.32
volumefrom_mean = 1.243764883985811E7
volumefrom_std = 2.930919029022742E7
volumefrom_min = 0.0
volumefrom_max = 2.30567037505E9
volumefrom_range = 2.305670375...


2.305670375...

In [13]:
// Saving data as Text

// don't forget to delete the previous folder 
// RDD does not allow us to overwrite: https://community.cloudera.com/t5/Support-Questions/Apache-SPARK-Overwrite-data-file/m-p/105253
val df_3 = df_2.withColumn("empty", typedLit(Seq(1)))
df_3.rdd.repartition(1).saveAsTextFile("data_cleaned")

println("All good")

All good


df_3 = [time: int, high: double ... 12 more fields]


[time: int, high: double ... 12 more fields]